# Extraccion de datos

In [5]:
import psycopg2
import pandas as pd
from query_sql import query_POSTGRESQL 
import os, sys

#%% RUTAS

path_files = os.getcwd()
# path_sql = os.path.join(os.path.normpath(os.getcwd() + os.sep + os.pardir), "funciones_sql") 
path_sql = os.path.join(os.path.normpath(os.getcwd() + os.sep + os.pardir), "Tarea2") 


#%% IMPORT DE FUNCIONES

sys.path.insert(0, path_sql)
from query_sql import query_POSTGRESQL, DataFrame2DataBase

#%% EXTRACCION DE DATOS

query_sql = '''SELECT * FROM dtes'''
data = query_POSTGRESQL(path_sql, 'USM', query_sql)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 535 entries, 0 to 534
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            535 non-null    int64 
 1   rutemisor     535 non-null    object
 2   folio         535 non-null    int64 
 3   date          535 non-null    object
 4   xml_dte       535 non-null    object
 5   convert_from  535 non-null    object
dtypes: int64(2), object(4)
memory usage: 25.2+ KB


# Transformacion de datos

In [ ]:
filas = []

for xml in data['convert_from']:

    root = ET.fromstring(xml)

    # Utiliza el namespace de XML
    namespace = {'default': 'http://www.sii.cl/SiiDte'}

    rut_emisor = root.find('.//default:RutEmisor', namespace).text

    razon_social = root.find('.//{http://www.sii.cl/SiiDte}RznSoc').text

    NmbItem = root.find('.//{http://www.sii.cl/SiiDte}NmbItem').text

    # a veces DscItem no existe
    DscItem = root.find('.//{http://www.sii.cl/SiiDte}DscItem')

    if DscItem is not None: # Crea nuevas filas para el df con DscItem
        nuevaFila = {
            "rut_emisor": rut_emisor,
            "razon_social": razon_social,
            # "NmbItem": NmbItem,
            # "DscItem": root.find('.//{http://www.sii.cl/SiiDte}DscItem').text,
            "Glosa": f'{NmbItem} {DscItem.text}'
        }
        filas.append(nuevaFila)
    else: # Crea nuevas filas para el df sin DscItem
        nuevaFila = {
            "rut_emisor": rut_emisor,
            "razon_social": razon_social,
            # "NmbItem": NmbItem,
            "Glosa": NmbItem,
        }
        filas.append(nuevaFila)

# agregar las nuevas filas a los distintos df
df_final = pd.DataFrame(filas)